In [1]:
import numpy as np
from math import floor
import music21
import re

def open_file(filename):
    # get the midi file from local
    mf=music21.midi.MidiFile()
    mf.open(fname)
    mf.read()
    mf.close()
    return mf


def get_stream(mf):
    # translate the midi file to stream
    midi_stream=music21.midi.translate.midiFileToStream(mf)
    return midi_stream

# Here we list some of the instruments which sound like violin or piano to add more data
VIOLINLIKE=["Violin", "Viola", "Cello", "Violincello", "Violoncello", "Flute", 
            "Oboe", "Clarinet", "Recorder", "Voice", "Piccolo",
            "StringInstrument", "Bassoon", "Horn"]
PIANOLIKE=["Piano", "Harp", "Harpsichord", "Organ", ""]


def assign_instrument(instr):
    # Determine if instrument is Piano-like or Violin-like
    if str(instr) in PIANOLIKE:
        return 0
    elif str(instr) in VIOLINLIKE:
        return 1
    else:
        print("Warning, unknown instrument: "+str(instr))
        return -1


def reassign_volume(velocity):
    #To simplify the volume question, we classify the volume into four subgroups  
    if 0 <= velocity <= 40:
        velocity = 35;
    if 40 < velocity <= 50:
        velocity = 45;
    if 50 < velocity <= 70:
        velocity = 60;
    if 70 < velocity <= 90:
        velocity =80;
    if 90 < velocity <= 100:
        velocity = 95;
    if 100 < velocity <= 127:
        velocity = 105;
    return(velocity)  

velocity = [0, 35, 45, 60, 80, 95, 105]
n = len(velocity)
vel_dic = dict(zip(list(range(n)),velocity))


def get_volume(velocity):
    for key, vel in vel_dic.items():
        if vel == velocity:
            return(key)
#to simplify the training process, we use the the number 0-8 to represent the real volume

def stream_to_chordwise(s, chamber, note_range, note_offset, sample_freq, velocity):  
    #sample_freq=4, note_range=62/38, note_offset=45, chamber=0, velocity = 1
    
    #we only use violin and piano in this case
    numInstruments=2 if chamber else 1 
    
    #the total timestep in a piece of music
    maxTimeStep = floor(s.duration.quarterLength * sample_freq)+1 
    
    # create empty 3D array to store note in each timestep
    score_arr = np.zeros((maxTimeStep, numInstruments, note_range)) 
    
    # create empty 2D array to store the volume in each timestep
    volume_arr = np.zeros((maxTimeStep, velocity)) 
    
    # create an empty list to store notes
    notes=[]
    
    # initialize instrumentID as zero
    instrumentID=0
    
    # we use filter to extract all the notes/ chords
    noteFilter=music21.stream.filters.ClassFilter('Note')
    chordFilter=music21.stream.filters.ClassFilter('Chord')
    
    #extract informaion from the note
    for n in s.recurse().addFilter(noteFilter):
        if chamber:
            instrumentID=assign_instrument(n.activeSite.getInstrument())
            if instrumentID==-1:
                return []
        volume = reassign_volume(n.volume.velocity)
        notes.append((n.pitch.midi-note_offset, floor(n.offset*sample_freq), floor(n.duration.quarterLength*sample_freq), instrumentID, volume))
    
    # pitch.midi-note_offset: pitch is the numerical representation of a note. 
    # note_offset is the the pitch relative to a zero mark. eg. B-=25, C=27, A=24

    # n.offset: the timestamps of each note, relative to the start of the score
    #           by multiplying with the sample_freq, you make all the timestamps integers

    # n.duration.quarterLength: the duration of that note as a float eg. quarter note = 0.25, half note = 0.5
    #                           multiply by sample_freq to represent duration in terms of timesteps
    # volume: the volume.velocity
    
    
    # extract information from chorder
    for c in s.recurse().addFilter(chordFilter):
        # unlike the noteFilter, this line of code is necessary as there are multiple notes in each chord
        # pitchesInChord is a list of notes at each chord eg. (<music21.pitch.Pitch D5>, <music21.pitch.Pitch F5>)
        pitchesInChord=c.pitches
        if chamber:
            instrumentID=assign_instrument(n.activeSite.getInstrument())     
            if instrumentID==-1:
                return []
        volume = reassign_volume(c.volume.velocity)
        for p in pitchesInChord:
            notes.append((p.midi-note_offset, floor(c.offset*sample_freq),floor(c.duration.quarterLength*sample_freq), instrumentID, volume))

    for n in notes:
        # pitch is the first variable in n, previously obtained by n.midi-note_offset
        pitch=n[0]
        
         # do some calibration for notes that fall our of note range
        # i.e. less than 0 or more than note_range
        while pitch<0:
            pitch+=12
        while pitch>=note_range:
            pitch-=12
        # the fourth element refers to instrument type => if instrument is violin, use different pitch calibration
        if n[3]==1:      #Violin lowest note is v22
            while pitch<22:
                pitch+=12
        
        # n[0] = pitch
        # n[1] = timestep
        # n[2] = duration
        # n[3] = instrument
        # n[4] = volume (used velocity to measure the amplitude on each Note or Pitch in chord.) range from 0-127
        score_arr[n[1], n[3], pitch]=1                  # Strike note
        score_arr[n[1]+1:n[1]+n[2], n[3], pitch]=2      # Continue holding note
        
        volume_arr[n[1],0] = n[4]                       # the volume of striking note 
        volume_arr[n[1]+1:n[1]+n[2], 0]= n[4]           # the volume of holding note
    
    #initialize an empty dictionary        
    instr={}
    
    instr[0]="p"
    instr[1]="v"
    
    #initialize an empty list to store the string of chord
    score_string_arr=[]
    for j in range(len(score_arr)):
        volume = str(get_volume(int(volume_arr[j][0])))
        # selecting the instruments: i=0 means piano and i=1 means violin
        for i in list(reversed(range(len(score_arr[j])))):   # List violin note first, then piano note
            prefix = volume +instr[i]
            score_string_arr.append(prefix+''.join([str(int(note)) for note in score_arr[j][i]]))    

    return score_string_arr

def add_modulations(score_string_arr):
    modulated=[]
    # get the note range from the array
    note_range=len(score_string_arr[0])-2
    
    for i in range(0,12):
        for chord in score_string_arr:
            # minus the instrument letter eg. 'p'
            # add 6 zeros on each side of the string
            padded='000000'+chord[2:]+'000000'
            # add back the instrument letter eg. 'p'
            # append window of len=note_range back into 
            # eg. if we have "00012345000"
            # iteratively, we want to get "p00012", "p00123", "p01234", "p12345", "p23450", "p34500", "p45000",
            modulated.append(chord[0:2]+padded[i:i+note_range])
    return modulated

def chord_to_notewise(long_string, sample_freq):
    translated_list=[]
    for j in range(len(long_string)):
    
        # chord at timestep j eg. 'p00000000000000000000000000000000000100'
        chord=long_string[j]
        next_chord=""
    
        # range is from next_timestep to max_timestep
        for k in range(j+1, len(long_string)):

            # checking if instrument of next chord is same as current chord
            if long_string[k][1]==chord[1]:

                # if same, set next chord as next element in modulation
                # otherwise, keep going until you find a chord with the same instrument
                # when you do, set it as the next chord
                next_chord=long_string[k]
                break

        # set prefix as the instrument
        # set chord and next_chord to be without the instrument prefix
        # next_chord is necessary to check when notes end
        volume=chord[0]
        instrument=chord[1]
        chord=chord[2:]
        next_chord=next_chord[2:]

        # checking for non-zero notes at one particular timestep
        # i is an integer indicating the index of each note the chord
        for i in range(len(chord)):

            if chord[i]=="0":
                continue

            # set note as 2 elements: instrument and index of note
            # examples: p22, p16, p4
            note=volume+instrument+str(i)                

            # if note in chord is 1, then append the note eg. p22 to the list
            if chord[i]=="1":
                translated_list.append(note)

            # If chord[i]=="2" do nothing - we're continuing to hold the note

            # unless next_chord[i] is back to "0" and it's time to end the note.
            if next_chord=="" or next_chord[i]=="0":      
                translated_list.append(volume+"end"+instrument+str(i))

        # wait indicates end of every timestep
        #if prefix=="p":
        translated_list.append("wait")

        # transform the list of notes into a string of notes
        # initialize i as zero and empty string
        i=0
        translated_string=""

    while i<len(translated_list):

        # stack all the repeated waits together using an integer to indicate the no. of waits
        # eg. "wait wait" => "wait2"
        wait_count=1
        if translated_list[i]=='wait':
            while wait_count<=sample_freq*2 and i+wait_count<len(translated_list) and translated_list[i+wait_count]=='wait':
                wait_count+=1
            translated_list[i]='wait'+str(wait_count)

        # add next note
        translated_string+=translated_list[i]+" "
        i+=wait_count
        
    return translated_string


def generate_music(fname,chordwise_folder,notewise_folder):
    sample_freq=4
    note_range=62
    note_offset=33
    chamber=0
    numInstruments=1
    velocity = 1
    
    # get midi file
    mf = open_file(fname)
    # get stream
    s = get_stream(mf)
    # chordwise encoding
    score_string_arr = stream_to_chordwise(s, chamber, note_range, note_offset, sample_freq, velocity)
    # modulate to get more data
    modulated = add_modulations(score_string_arr)
    # notewise encoding
    translated_string = chord_to_notewise(modulated, sample_freq)
    
    # export chordwise encoding
    
    pattern = re.compile('.*?/(.*?).mid')
    chordname = re.findall(pattern, fname)[0]
    notename = re.findall(pattern, fname)[0]
    
    f=open(chordwise_folder+chordname+".txt","w+")   
    f.write(" ".join(modulated))
    f.close()
    
    # export notewise encoding
    f=open(notewise_folder+notename+".txt","w+")
    f.write(translated_string)
    f.close()
    
fname = "midi-files/sonat-2.mid"
chordwise_folder = "./txt-files/chordwise/"
notewise_folder = "./txt-files/notewise/"

generate_music(fname,chordwise_folder,notewise_folder)


In [4]:
import os
midi_files = os.listdir('/Users/chenyishuang/Desktop/encoding-decoding/midi-files')

In [7]:
chordwise_folder = "./txt-files/chordwise/"
notewise_folder = "./txt-files/notewise/"
for i in range(len(midi_files)):
    fname = 'midi-files/'+midi_files[i]
    generate_music(fname,chordwise_folder,notewise_folder)